In [ ]:
def pid_flap_control(Kp, Ki, Kd, error, prev_error, integral, dt, current_action, output_limits=(-2, 2)):
    """
    PID controller for incremental adjustments (flap motion).
    
    Kp, Ki, Kd: PID coefficients.
    setpoint: Desired target value.
    current_value: Current value of the system.
    prev_error: Error from the previous time step.
    integral: Accumulated integral of error.
    dt: Time step.
    current_action: The current action value that is incrementally changed.
    output_limits: Limits for the output (min, max).
    """
    # Calculate error
    
    # Proportional term
    P = Kp * error
    
    # Integral term
    integral += error * dt
    I = Ki * integral
    
    # Derivative term
    derivative = (error - prev_error) / dt
    D = Kd * derivative
    
    # PID output (this is the incremental change, not the absolute value)
    delta_action = P + I + D
    
    # Adjust the current action based on the delta from the PID output
    new_action = current_action + delta_action
    
    # Apply output limits
    new_action = max(min(new_action, output_limits[1]), output_limits[0])
    
    # Return the updated action, error, and integral for the next time step
    return new_action, error, integral

# Example usage:
# PID parameters
Kp = 1.0
Ki = 0.1
Kd = 0.05

# System parameters
setpoint = 10       # Desired value
current_value = 0   # Initial system value
prev_error = 0      # Initialize previous error
integral = 0        # Initialize integral
current_action = 0  # Initial action (e.g., no motion)
dt = 0.2            # Time step (in seconds)

# Example step in control loop
new_action, error, integral = pid_flap_control(Kp, Ki, Kd, setpoint, current_value, prev_error, integral, dt, current_action)

print(f"Updated Action: {new_action}")


In [ ]:
import gym
import gym_fish
import numpy as np

# Our environment runs on GPU to accelerate simulations, ensure a cuda-supported GPU exists on your machine
gpuId = 0
env = gym.make('koi-cruising-v0', gpuId=gpuId)

def pid_flap_control(Kp, Ki, Kd, error, prev_error, integral, dt, output_limits=(-2, 2)):
    """
    PID controller for incremental adjustments (flap motion).
    
    Kp, Ki, Kd: PID coefficients.
    setpoint: Desired target value.
    current_value: Current value of the system.
    prev_error: Error from the previous time step.
    integral: Accumulated integral of error.
    dt: Time step.
    current_action: The current action value that is incrementally changed.
    output_limits: Limits for the output (min, max).
    """
    # Calculate error
    
    # Proportional term
    P = Kp * error
    
    # Integral term
    integral += error * dt
    I = Ki * integral
    
    # Derivative term
    derivative = (error - prev_error) / dt
    D = Kd * derivative
    
    # PID output (this is the incremental change, not the absolute value)
    delta_action = P + I + D

    # new_action=
    
    # Adjust the current action based on the delta from the PID output
    
    # Apply output limits
    # new_action = max(min(new_action, output_limits[1]), output_limits[0])
    
    # Return the updated action, error, and integral for the next time step
    return delta_action, error, integral

# Example usage:
# PID parameters
Kp = 1.5
Ki = 0.1
Kd = 0.05

# System parameters
setpoint = 10       # Desired value
old_action = 0   # Initial system value
prev_error = 0      # Initialize previous error
integral_old = 0        # Initialize integral
dt = 0.2            # Time step (in seconds)

# Example step in control loop
curr_err=np.linalg.norm((env.goal_pos-env.body_xyz))

prev_error=curr_err

i=1
with open("body_xyz_values", 'w') as body_xyz, open("error.txt","w") as error_file :

    while np.linalg.norm((env.goal_pos-env.body_xyz)) <  15 : 
        
        curr_err=np.linalg.norm((env.goal_pos-env.body_xyz))
        delta_action, error, integral_new = pid_flap_control(Kp, Ki, Kd, curr_err, prev_error, integral_old, dt)
        
        action=old_action+delta_action
        if action >=2 or action <= -2:
            action=0


        obs, reward, done, info = env.step([action])

        body_xyz.write(f"{env.body_xyz}/n")
        error_file.write(f"{curr_err}\n")
        print("steps",i, "error",curr_err)

        # print("action",action)
        # print("prev_error",prev_error)
        # print("integral",integral_new)

        prev_error=curr_err
        integral_old=integral_new
        old_action=action
        i+=1





##### MPC control

In [ ]:
import gym
import gym_fish
import numpy as np

# Our environment runs on GPU to accelerate simulations, ensure a cuda-supported GPU exists on your machine
gpuId = 0
env = gym.make('koi-cruising-v0', gpuId=gpuId)


i=1
with open("body_xyz_values", 'w') as body_xyz, open("error.txt","w") as error_file :

    while np.linalg.norm((env.goal_pos-env.body_xyz)) <  15 : 
        curr_err=np.linalg.norm((env.goal_pos-env.body_xyz))

        action=env.action_space.sample()
        obs, reward, done, info = env.step(action)

        body_xyz.write(f"{env.body_xyz}/n")
        error_file.write(f"{curr_err}\n")
        print("steps",i, "error",curr_err)
        


